In [1]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

In [2]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 9830684516775003347, name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 9214062756
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 232423787903818629
 physical_device_desc: "device: 0, name: GeForce GTX 1080 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1"]

In [3]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials, partial, rand, space_eval
from sklearn.metrics import log_loss
import sys
import time
import math
import os
import pandas as pd
from keras.callbacks import EarlyStopping
import numpy as np
from glob import glob
import cv2
import skimage
from skimage.transform import resize
from keras.utils.np_utils import to_categorical
import keras
from keras import layers
from keras import models
from keras import optimizers
from keras.models import load_model
# import keras.callbacks as kcall
from keras.optimizers import Adam, RMSprop,SGD
from keras.models import Model
from keras.models import Sequential
from keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D
from keras.applications.xception import Xception, preprocess_input
from keras.preprocessing.image import ImageDataGenerator
# from keras.callbacks import EarlyStopping, TensorBoard, ModelCheckpoint
from keras.applications.vgg19 import VGG19
from keras.regularizers import l2

import matplotlib.pyplot as plt
from keras.layers import Input, concatenate
from keras import optimizers, metrics, models
from keras.layers import Input, Flatten, Dense

%matplotlib inline

Using TensorFlow backend.


In [4]:
import keras
import tensorflow as tf

print("Keras Version", keras.__version__)
print("tensorflow Version", tf.__version__)
# print("dim_ordering:", K.image_dim_ordering())

Keras Version 2.2.4
tensorflow Version 1.9.0


In [5]:
batch_size = 32
img_height, img_width = 128, 128
input_shape = (img_height, img_width, 3)
epochs = 15

In [6]:
train_dir = 'data/train/'
test_dir = 'data/test'

In [7]:
random_seed = np.random.seed(1142)

train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    featurewise_center=True,
    featurewise_std_normalization=True,
    validation_split= 0.3,
    zoom_range=0.2,
    shear_range=0.2)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'training',
    class_mode='categorical')

validation_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    subset = 'validation',
    class_mode='categorical')

test_datagen = ImageDataGenerator(rescale=1. / 255)
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    seed = random_seed,
    shuffle = False,
    class_mode='categorical')

Found 34909 images belonging to 6 classes.
Found 14958 images belonging to 6 classes.
Found 13578 images belonging to 6 classes.


In [8]:
nb_train_samples = len(train_generator.filenames)
nb_validation_samples = len(validation_generator.filenames)
nb_test_samples = len(test_generator.filenames)

predict_size_train = int(math.ceil(nb_train_samples / batch_size))
predict_size_validation = int(math.ceil(nb_validation_samples / batch_size))
predict_size_test = int(math.ceil(nb_test_samples / batch_size))

num_classes = len(train_generator.class_indices)

print("nb_train_samples:", nb_train_samples)
print("nb_validation_samples:", nb_validation_samples)
print("nb_test_samples:", nb_test_samples)

print("\npredict_size_train:", predict_size_train)
print("predict_size_validation:", predict_size_validation)
print("predict_size_test:", predict_size_test)

print("\n num_classes:", num_classes)

nb_train_samples: 34909
nb_validation_samples: 14958
nb_test_samples: 13578

predict_size_train: 1091
predict_size_validation: 468
predict_size_test: 425

 num_classes: 6


In [9]:
# vgg19_weights ="../input/full-keras-pretrained-no-top/vgg19_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_weights ="../input/full-keras-pretrained-no-top//inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5"
# vgg16_weights ="../input/full-keras-pretrained-no-top/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet201_weights ="../input/full-keras-pretrained-no-top/densenet201_weights_tf_dim_ordering_tf_kernels_notop.h5"
# denseNet121_weights ="../input/full-keras-pretrained-no-top/densenet121_weights_tf_dim_ordering_tf_kernels_notop.h5"
# resenet50_weights ="../input/full-keras-pretrained-no-top/resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
# inception_resnet_v2_weights ="../input/full-keras-pretrained-no-top/inception_resnet_v2_weights_tf_dim_ordering_tf_kernels_notop.h5"
# nasnet_weights ="../input/full-keras-pretrained-no-top/nasnet_large_no_top.h5"
# nasnet_mobile_weights ="../input/full-keras-pretrained-no-top/nasnet_mobile_no_top.h5"
# mobilenet_weights ="../input/full-keras-pretrained-no-top/mobilenet_1_0_224_tf_no_top.h5"

In [10]:
# from keras.applications.vgg19 import VGG19
# from keras.applications.inception_v3 import InceptionV3
# from keras.applications.vgg16 import VGG16
from keras.applications.xception import Xception, preprocess_input
# from keras.applications import DenseNet201
# from keras.applications import DenseNet121
# from keras.applications import ResNet50
# from keras.applications.inception_resnet_v2 import InceptionResNetV2
# from keras.applications import NASNetLarge, NASNetMobile
# from keras.applications import MobileNet

bottleneck_final_model=Xception(weights="imagenet", include_top=False, pooling = "avg")

In [11]:
# os.mkdir("extracted_features")
extracted_features_dir = "extracted_features/"
model_name = "Xception_descriptors"

In [12]:
bottleneck_features_train = bottleneck_final_model.predict_generator(train_generator, predict_size_train, max_q_size=1, pickle_safe=False)
np.save(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy', bottleneck_features_train)

In [13]:
bottleneck_features_validation = bottleneck_final_model.predict_generator(validation_generator, predict_size_validation)
np.save(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy', bottleneck_features_validation)

In [14]:
bottleneck_features_test = bottleneck_final_model.predict_generator(test_generator, predict_size_test)
np.save(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy', bottleneck_features_test)

In [15]:

import tensorflow as tf
from keras.backend.tensorflow_backend import get_session
from keras.backend.tensorflow_backend import clear_session
from keras.backend.tensorflow_backend import set_session

def reset_keras_tf_session():
    """
    this function clears the gpu memory and set the 
    tf session to not use the whole gpu
    """
    sess = get_session()
    clear_session()
    sess.close()
    sess = get_session()

#     config = tf.ConfigProto()
#     config.gpu_options.allow_growth = True
#     set_session(tf.Session(config=config))

reset_keras_tf_session()

In [16]:
train_data = np.load(extracted_features_dir+'bottleneck_features_train_'+model_name+'.npy')
validation_data = np.load(extracted_features_dir+'bottleneck_features_validation_'+model_name+'.npy')
test_data = np.load(extracted_features_dir+'bottleneck_features_test_'+model_name+'.npy')

train_labels = train_generator.classes
train_labels = to_categorical(train_labels, num_classes=num_classes)

validation_labels = validation_generator.classes
validation_labels = to_categorical(validation_labels, num_classes=num_classes)

test_labels = test_generator.classes
test_labels = to_categorical(test_labels, num_classes=num_classes)

In [17]:
space = {
         'lr': hp.choice('lr',[0.1, 0.01, 0.001, 0.0001, 0.00001,0.000001, 0.0000001]),
#          'dropout': hp.choice('dropout', [0.4, 0.5, 0.6, 0.7]),
#          'batch_size': hp.choice('batch_size', [64]),
#          'epochs': hp.choice('epochs', [15, 20, 25, 30, 50]),
#          'optimizer': hp.choice('optimizer',['sgd','adam','rmsprop']),
#          'optimizer': hp.choice('optimizer',['rmsprop']),
#          'optimizer': hp.choice('optimizer',['adam']),
         'beta_1':hp.choice('beta_1',[0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8, 0.9]),
         'beta_2':hp.choice('beta_2',[0.3,0.4,0.5,0.6,0.7,0.8,0.9,0.999,0.99,0.995]),
#          'momentum':hp.choice('momentum',[0.3,0.5,0.7,0.9,1]),
#          'amsgrad':hp.choice('amsgrad',[False,True]),
#          'nesterov':hp.choice('nesterov',[False,True]),
#          'rho':hp.choice('rho',[0.4,0.5,0.6,0.7,0.8,0.9,1]),
        }

In [18]:
from sklearn.metrics import accuracy_score
def f_nn(params):   
    print ('Parameters testing: ', params)
  
    adam_opt=Adam(lr=params["lr"], beta_1=params["beta_1"], beta_2=params['beta_2'], epsilon=None, decay=0.0)
#     sgd=SGD(lr=params["lr"], momentum=params['momentum'], decay=0.0, nesterov=params['nesterov'])
#     rmsprop=RMSprop(lr=params["lr"], rho=params['rho'], epsilon=None, decay=0.0)

    model = Sequential()
    # model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(0.7))
    model.add(Dense(num_classes, activation='softmax'))

    model.compile(optimizer=adam_opt, loss='categorical_crossentropy', metrics=['accuracy'])
    history = model.fit(train_data, train_labels,
                        epochs=epochs,
                        batch_size=batch_size,
                        validation_data=(validation_data, validation_labels),
                        verbose= 2,
                        callbacks=get_callbacks(params))

    (eval_loss, eval_accuracy) = model.evaluate(validation_data, validation_labels, batch_size= batch_size, verbose=1)

    print("Validation Accuracy: {:.4f}%".format(eval_accuracy * 100))
    print("Validation Loss: {}".format(eval_loss))
    
    filename = test_generator.filenames
    truth = test_generator.classes
    label = test_generator.class_indices
    indexlabel = dict((value, key) for key, value in label.items())

    preds = model.predict(test_data)

    predictions = [i.argmax() for i in preds]
    y_true = [i.argmax() for i in test_labels]
#     cm = confusion_matrix(y_pred=predictions, y_true=y_true)

    print('Test Accuracy: {}'.format(accuracy_score(y_true=y_true, y_pred=predictions)))

    print("*_*" * 50)
#     best_epoch = np.argmax(history.history['val_acc'])
#     best_val_acc = np.max(history.history['val_acc'])
#     print('Epoch {} - val acc: {}'.format(best_epoch, best_val_acc))
    sys.stdout.flush() 
    
    return {'loss': eval_loss, 'status': STATUS_OK, 'model': model}

In [19]:
def get_callbacks(params):
    callbacks =[EarlyStopping(monitor='val_acc', patience=3, verbose=1)]
    return callbacks

In [20]:
trials = Trials()
best = fmin(f_nn, space, algo=tpe.suggest, max_evals = 300, trials=trials)
print(best)

Parameters testing:  {'beta_1': 0.2, 'beta_2': 0.8, 'lr': 0.001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.6945 - acc: 0.7687 - val_loss: 1.1176 - val_acc: 0.6447
Epoch 2/15
 - 4s - loss: 0.5650 - acc: 0.8235 - val_loss: 1.0201 - val_acc: 0.6732
Epoch 3/15
 - 4s - loss: 0.5503 - acc: 0.8409 - val_loss: 1.1466 - val_acc: 0.6699
Epoch 4/15
 - 4s - loss: 0.5430 - acc: 0.8497 - val_loss: 1.1539 - val_acc: 0.6842
Epoch 5/15
 - 4s - loss: 0.5562 - acc: 0.8545 - val_loss: 1.1060 - val_acc: 0.6925
Epoch 6/15
 - 4s - loss: 0.5698 - acc: 0.8576 - val_loss: 1.3871 - val_acc: 0.6724
Epoch 7/15
 - 4s - loss: 0.5767 - acc: 0.8598 - val_loss: 1.2367 - val_acc: 0.6936
Epoch 8/15
 - 4s - loss: 0.5746 - acc: 0.8636 - val_loss: 1.2589 - val_acc: 0.6991
Epoch 9/15
 - 4s - loss: 0.5883 - acc: 0.8633 - val_loss: 1.3481 - val_acc: 0.6899
Epoch 10/15
 - 4s - loss: 0.5904 - acc: 0.8644 - val_loss: 1.4407 - val_acc: 0.6913
Epoch 11/15
 - 4s - loss: 0.5957 - acc: 0.8658 - val_l

Parameters testing:  {'beta_1': 0.9, 'beta_2': 0.5, 'lr': 1e-07}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.9381 - acc: 0.1704 - val_loss: 1.8243 - val_acc: 0.1857
Epoch 2/15
 - 4s - loss: 1.8892 - acc: 0.1913 - val_loss: 1.8118 - val_acc: 0.2157
Epoch 3/15
 - 4s - loss: 1.8440 - acc: 0.2162 - val_loss: 1.8003 - val_acc: 0.2431
Epoch 4/15
 - 4s - loss: 1.8107 - acc: 0.2309 - val_loss: 1.7899 - val_acc: 0.2654
Epoch 5/15
 - 4s - loss: 1.7738 - acc: 0.2527 - val_loss: 1.7803 - val_acc: 0.2813
Epoch 6/15
 - 4s - loss: 1.7415 - acc: 0.2764 - val_loss: 1.7713 - val_acc: 0.2945
Epoch 7/15
 - 4s - loss: 1.7071 - acc: 0.2987 - val_loss: 1.7628 - val_acc: 0.3048
Epoch 8/15
 - 4s - loss: 1.6802 - acc: 0.3177 - val_loss: 1.7548 - val_acc: 0.3131
Epoch 9/15
 - 4s - loss: 1.6530 - acc: 0.3377 - val_loss: 1.7470 - val_acc: 0.3208
Epoch 10/15
 - 4s - loss: 1.6313 - acc: 0.3539 - val_loss: 1.7395 - val_acc: 0.3286
Epoch 11/15
 - 4s - loss: 1.6071 - acc: 0.3703 - val_l

Parameters testing:  {'beta_1': 0.7, 'beta_2': 0.99, 'lr': 1e-05}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 1.3874 - acc: 0.4934 - val_loss: 1.4801 - val_acc: 0.3922
Epoch 2/15
 - 4s - loss: 1.0726 - acc: 0.6522 - val_loss: 1.3451 - val_acc: 0.4081
Epoch 3/15
 - 4s - loss: 0.9496 - acc: 0.6777 - val_loss: 1.2787 - val_acc: 0.4207
Epoch 4/15
 - 4s - loss: 0.8788 - acc: 0.6940 - val_loss: 1.2408 - val_acc: 0.4302
Epoch 5/15
 - 4s - loss: 0.8305 - acc: 0.7057 - val_loss: 1.1993 - val_acc: 0.4471
Epoch 6/15
 - 4s - loss: 0.7935 - acc: 0.7165 - val_loss: 1.1775 - val_acc: 0.4547
Epoch 7/15
 - 4s - loss: 0.7651 - acc: 0.7243 - val_loss: 1.1536 - val_acc: 0.4670
Epoch 8/15
 - 4s - loss: 0.7422 - acc: 0.7333 - val_loss: 1.1261 - val_acc: 0.4815
Epoch 9/15
 - 4s - loss: 0.7202 - acc: 0.7412 - val_loss: 1.1070 - val_acc: 0.4898
Epoch 10/15
 - 4s - loss: 0.7039 - acc: 0.7473 - val_loss: 1.0870 - val_acc: 0.5025
Epoch 11/15
 - 4s - loss: 0.6916 - acc: 0.7525 - val_

 - 4s - loss: 0.4164 - acc: 0.8573 - val_loss: 0.8773 - val_acc: 0.7018
Epoch 6/15
 - 4s - loss: 0.3956 - acc: 0.8634 - val_loss: 0.9274 - val_acc: 0.6814
Epoch 7/15
 - 4s - loss: 0.3844 - acc: 0.8689 - val_loss: 1.0589 - val_acc: 0.6775
Epoch 8/15
 - 4s - loss: 0.3774 - acc: 0.8714 - val_loss: 0.9426 - val_acc: 0.6939
Epoch 00008: early stopping
14958/14958 [==============================] - 1s 39us/step
Validation Accuracy: 69.3876%
Validation Loss: 0.9425617923864454
Test Accuracy: 0.8252319929297393
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.8, 'beta_2': 0.4, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 5s - loss: 0.8249 - acc: 0.7099 - val_loss: 1.0573 - val_acc: 0.5518
Epoch 2/15
 - 4s - loss: 0.6121 - acc: 0.7885 - val_loss: 0.9818 - val_acc: 0.6104
Epoch 3/15
 - 4s - loss: 0.5520 - acc: 0.8101 - val_loss: 

Epoch 2/15
 - 4s - loss: 1.0554 - acc: 0.6538 - val_loss: 1.3483 - val_acc: 0.4023
Epoch 3/15
 - 4s - loss: 0.9405 - acc: 0.6808 - val_loss: 1.2754 - val_acc: 0.4248
Epoch 4/15
 - 4s - loss: 0.8732 - acc: 0.6975 - val_loss: 1.2325 - val_acc: 0.4410
Epoch 5/15
 - 4s - loss: 0.8304 - acc: 0.7078 - val_loss: 1.2065 - val_acc: 0.4546
Epoch 6/15
 - 4s - loss: 0.7984 - acc: 0.7194 - val_loss: 1.1727 - val_acc: 0.4719
Epoch 7/15
 - 4s - loss: 0.7700 - acc: 0.7281 - val_loss: 1.1578 - val_acc: 0.4834
Epoch 8/15
 - 4s - loss: 0.7533 - acc: 0.7363 - val_loss: 1.1424 - val_acc: 0.4936
Epoch 9/15
 - 4s - loss: 0.7346 - acc: 0.7407 - val_loss: 1.1273 - val_acc: 0.5078
Epoch 10/15
 - 4s - loss: 0.7167 - acc: 0.7497 - val_loss: 1.1145 - val_acc: 0.5184
Epoch 11/15
 - 4s - loss: 0.7013 - acc: 0.7567 - val_loss: 1.1038 - val_acc: 0.5287
Epoch 12/15
 - 4s - loss: 0.6935 - acc: 0.7611 - val_loss: 1.0898 - val_acc: 0.5409
Epoch 13/15
 - 4s - loss: 0.6837 - acc: 0.7645 - val_loss: 1.0767 - val_acc: 0.5528


...

Epoch 7/15
 - 10s - loss: 1.9570 - acc: 0.7695 - val_loss: 2.7455 - val_acc: 0.6685
Epoch 8/15
 - 9s - loss: 2.0170 - acc: 0.7715 - val_loss: 2.6616 - val_acc: 0.6828
Epoch 9/15
 - 10s - loss: 2.0454 - acc: 0.7708 - val_loss: 2.6843 - val_acc: 0.6594
Epoch 10/15
 - 10s - loss: 2.0727 - acc: 0.7660 - val_loss: 2.9965 - val_acc: 0.6561
Epoch 11/15
 - 10s - loss: 2.0693 - acc: 0.7727 - val_loss: 3.3721 - val_acc: 0.6387
Epoch 00011: early stopping
14958/14958 [==============================] - 2s 135us/step
Validation Accuracy: 63.8655%
Validation Loss: 3.372129678351236
Test Accuracy: 0.7699955810870526
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.4, 'beta_2': 0.995, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 21s - loss: 0.9015 - acc: 0.6844 - val_loss: 1.1142 - val_acc: 0.4856
Epoch 2/15
 - 9s - loss: 0.6651 - acc:

Test Accuracy: 0.7852408307556341
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.6, 'beta_2': 0.999, 'lr': 0.1}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 22s - loss: 11.5001 - acc: 0.2838 - val_loss: 10.4602 - val_acc: 0.3504
Epoch 2/15
 - 12s - loss: 10.2175 - acc: 0.3659 - val_loss: 11.0684 - val_acc: 0.3131
Epoch 3/15
 - 9s - loss: 9.9271 - acc: 0.3840 - val_loss: 11.4999 - val_acc: 0.2862
Epoch 4/15
 - 9s - loss: 10.6603 - acc: 0.3384 - val_loss: 11.6111 - val_acc: 0.2795
Epoch 00004: early stopping
14958/14958 [==============================] - 2s 141us/step
Validation Accuracy: 27.9516%
Validation Loss: 11.611121710552732
Test Accuracy: 0.35314479304757695
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing

Test Accuracy: 0.7898806893504198
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'beta_1': 0.6, 'beta_2': 0.8, 'lr': 0.01}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 23s - loss: 1.0904 - acc: 0.7298 - val_loss: 1.1559 - val_acc: 0.6820
Epoch 2/15
 - 12s - loss: 1.1638 - acc: 0.7689 - val_loss: 1.7458 - val_acc: 0.6524
Epoch 3/15
 - 11s - loss: 1.3345 - acc: 0.7787 - val_loss: 1.7704 - val_acc: 0.6679
Epoch 4/15
 - 12s - loss: 1.3832 - acc: 0.7885 - val_loss: 1.9017 - val_acc: 0.6816
Epoch 00004: early stopping
14958/14958 [==============================] - 3s 175us/step
Validation Accuracy: 68.1642%
Validation Loss: 1.9016956842784283
Test Accuracy: 0.7955516276329356
*_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_**_*
Parameters testing:  {'be

Epoch 5/15
 - 9s - loss: 0.5242 - acc: 0.8166 - val_loss: 0.9238 - val_acc: 0.6310
Epoch 6/15
 - 9s - loss: 0.5000 - acc: 0.8231 - val_loss: 0.9253 - val_acc: 0.6415
Epoch 7/15
 - 10s - loss: 0.4804 - acc: 0.8309 - val_loss: 0.8823 - val_acc: 0.6473
Epoch 8/15
 - 10s - loss: 0.4625 - acc: 0.8350 - val_loss: 0.9190 - val_acc: 0.6455
Epoch 9/15
 - 11s - loss: 0.4490 - acc: 0.8408 - val_loss: 0.9078 - val_acc: 0.6543
Epoch 10/15
 - 10s - loss: 0.4342 - acc: 0.8465 - val_loss: 0.9041 - val_acc: 0.6558
Epoch 11/15
 - 9s - loss: 0.4248 - acc: 0.8492 - val_loss: 0.8891 - val_acc: 0.6623
Epoch 12/15
 - 9s - loss: 0.4135 - acc: 0.8518 - val_loss: 0.8869 - val_acc: 0.6633
Epoch 13/15
 - 11s - loss: 0.4051 - acc: 0.8559 - val_loss: 0.8725 - val_acc: 0.6732
Epoch 14/15
 - 10s - loss: 0.3965 - acc: 0.8579 - val_loss: 0.8818 - val_acc: 0.6708
Epoch 15/15
 - 10s - loss: 0.3880 - acc: 0.8604 - val_loss: 0.9056 - val_acc: 0.6671
14958/14958 [==============================] - 2s 144us/step
Validation Ac

Parameters testing:  {'beta_1': 0.4, 'beta_2': 0.999, 'lr': 0.0001}
Train on 34909 samples, validate on 14958 samples
Epoch 1/15
 - 23s - loss: 0.9105 - acc: 0.6786 - val_loss: 1.0988 - val_acc: 0.4971
Epoch 2/15
 - 12s - loss: 0.6686 - acc: 0.7630 - val_loss: 1.0148 - val_acc: 0.5794
Epoch 3/15
 - 11s - loss: 0.6026 - acc: 0.7897 - val_loss: 1.0020 - val_acc: 0.5952
Epoch 4/15
 - 10s - loss: 0.5536 - acc: 0.8078 - val_loss: 0.9345 - val_acc: 0.6256
Epoch 5/15
 - 10s - loss: 0.5231 - acc: 0.8175 - val_loss: 0.9327 - val_acc: 0.6310
Epoch 6/15
 - 10s - loss: 0.4985 - acc: 0.8235 - val_loss: 0.9228 - val_acc: 0.6385
Epoch 7/15
 - 10s - loss: 0.4788 - acc: 0.8308 - val_loss: 0.9097 - val_acc: 0.6471
Epoch 8/15
 - 11s - loss: 0.4640 - acc: 0.8384 - val_loss: 0.8851 - val_acc: 0.6598
Epoch 9/15
 - 10s - loss: 0.4470 - acc: 0.8412 - val_loss: 0.9137 - val_acc: 0.6487
Epoch 10/15
 - 10s - loss: 0.4318 - acc: 0.8464 - val_loss: 0.8885 - val_acc: 0.6597
Epoch 11/15
 - 10s - loss: 0.4215 - acc: 